In [29]:
import numpy as np 
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score


In [263]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
boston = load_boston()
X, Y = boston.data, boston.target
#X = X[0:130,:]
#Y = (Y[Y != 2])
Y = Y[:,np.newaxis]
#split data into training and test data.
#train_X, test_X, train_y, test_y = train_test_split(X, Y, train_size=0.9,test_size=0.1)
#train_y
#train_y
print(Y.shape)
X = X[:,9:10]
print(Y)
print(X.shape)

(506, 1)
[[24. ]
 [21.6]
 [34.7]
 [33.4]
 [36.2]
 [28.7]
 [22.9]
 [27.1]
 [16.5]
 [18.9]
 [15. ]
 [18.9]
 [21.7]
 [20.4]
 [18.2]
 [19.9]
 [23.1]
 [17.5]
 [20.2]
 [18.2]
 [13.6]
 [19.6]
 [15.2]
 [14.5]
 [15.6]
 [13.9]
 [16.6]
 [14.8]
 [18.4]
 [21. ]
 [12.7]
 [14.5]
 [13.2]
 [13.1]
 [13.5]
 [18.9]
 [20. ]
 [21. ]
 [24.7]
 [30.8]
 [34.9]
 [26.6]
 [25.3]
 [24.7]
 [21.2]
 [19.3]
 [20. ]
 [16.6]
 [14.4]
 [19.4]
 [19.7]
 [20.5]
 [25. ]
 [23.4]
 [18.9]
 [35.4]
 [24.7]
 [31.6]
 [23.3]
 [19.6]
 [18.7]
 [16. ]
 [22.2]
 [25. ]
 [33. ]
 [23.5]
 [19.4]
 [22. ]
 [17.4]
 [20.9]
 [24.2]
 [21.7]
 [22.8]
 [23.4]
 [24.1]
 [21.4]
 [20. ]
 [20.8]
 [21.2]
 [20.3]
 [28. ]
 [23.9]
 [24.8]
 [22.9]
 [23.9]
 [26.6]
 [22.5]
 [22.2]
 [23.6]
 [28.7]
 [22.6]
 [22. ]
 [22.9]
 [25. ]
 [20.6]
 [28.4]
 [21.4]
 [38.7]
 [43.8]
 [33.2]
 [27.5]
 [26.5]
 [18.6]
 [19.3]
 [20.1]
 [19.5]
 [19.5]
 [20.4]
 [19.8]
 [19.4]
 [21.7]
 [22.8]
 [18.8]
 [18.7]
 [18.5]
 [18.3]
 [21.2]
 [19.2]
 [20.4]
 [19.3]
 [22. ]
 [20.3]
 [20.5]
 [17.3]

In [286]:


def rbf(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)
    
def kmeans(X, k):
  
 
    # randomly select initial clusters from input data
    clusters = np.random.choice(np.squeeze(X), size=k)
    prevClusters = clusters.copy()
    stds = np.zeros(k)
    converged = False
 
    while not converged:
       
        distances = np.squeeze(np.abs(X[:, np.newaxis] - clusters[np.newaxis, :]))
 
        # find the cluster that's closest to each point
        closestCluster = np.argmin(distances, axis=1)
 
        # update clusters by taking the mean of all of the points assigned to that cluster
        for i in range(k):
            pointsForCluster = X[closestCluster == i]
            if len(pointsForCluster) > 0:
                clusters[i] = np.mean(pointsForCluster, axis=0)
 
        # converge if clusters haven't moved
        converged = np.linalg.norm(clusters - prevClusters) < 1e-6
        prevClusters = clusters.copy()
 
    distances = np.squeeze(np.abs(X[:, np.newaxis] - clusters[np.newaxis, :]))
    closestCluster = np.argmin(distances, axis=1)
 
    clustersWithNoPoints = []
    for i in range(k):
        pointsForCluster = X[closestCluster == i]
        if len(pointsForCluster) < 2:
            # keep track of clusters with no points or 1 point
            clustersWithNoPoints.append(i)
            continue
        else:
            stds[i] = np.std(X[closestCluster == i])
 
    # if there are clusters with 0 or 1 points, take the mean std of the other clusters
    if len(clustersWithNoPoints) > 0:
        pointsToAverage = []
        for i in range(k):
            if i not in clustersWithNoPoints:
                pointsToAverage.append(X[closestCluster == i])
        pointsToAverage = np.concatenate(pointsToAverage).ravel()
        stds[clustersWithNoPoints] = np.mean(np.std(pointsToAverage))
 
    return clusters, stds
    
    


In [322]:

class RBFNet(object):
    
    def __init__(self, k=2, lr=0.01, epochs=100, rbf=rbf, inferStds=True):
        self.k = k
        self.lr = lr
        self.epochs = epochs
        self.rbf = rbf
        self.inferStds = inferStds

        self.w = np.random.randn(k)
        self.b = np.random.randn(1)
    
    
    def fit(self, X, y):
            if self.inferStds:
                    # compute stds from data
                self.centers, self.stds = kmeans(X, self.k)
            else:
                # use a fixed std 
                self.centers, _ = kmeans(X, self.k)
                dMax = max([np.abs(c1 - c2) for c1 in self.centers for c2 in self.centers])
                self.stds = np.repeat(dMax / np.sqrt(2*self.k), self.k)

            # training
            for epoch in range(self.epochs):
                for i in range(X.shape[0]):
                    # forward pass
                    a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                    F = a.T.dot(self.w) + self.b

                    loss = (y[i] - F).flatten() ** 2
                    #print('Loss: {0:.2f}'.format(loss[0]))

                    # backward pass
                    error = -(y[i] - F).flatten()

                    # online update
                    self.w = self.w - self.lr * a * error
                    self.b = self.b - self.lr * error


    def predict(self, X):
        y_pred = []
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w) + self.b
            y_pred.append(F)
        return np.array(y_pred)

In [323]:
rbfnet = RBFNet(lr=1e-2, k=2)

rbfnet.fit(X, Y)
 
y_pred = rbfnet.predict(X)

In [326]:
print(y_pred.shape)
y_pred = y_pred[:,:,0]
print(y_pred.shape)
print(y_pred)


(506, 1, 2)
(506, 1)
[[23.09858164]
 [23.9340465 ]
 [23.9340465 ]
 [24.30794375]
 [24.30794375]
 [24.30794375]
 [23.03974405]
 [23.03974405]
 [23.03974405]
 [23.03974405]
 [23.03974405]
 [23.03974405]
 [23.03974405]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.04526192]
 [23.28085144]
 [23.28085144]
 [23.28085144]
 [23.28085144]
 [23.74084776]
 [23.74084776]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [24.10668817]
 [23.91466859]
 [23.91466859]
 [23.91466859]
 [23.91466859]
 [25.05781377]
 [24.23647534]
 [23.03981311]
 [23.66500784]
 [23.21598139]
 [23.21598139]
 [23.21598139]
 [23.21598139]
 [23.21598139]
 [23.21598139]
 [24.41059231]
 [23

In [327]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [329]:

print(mean_squared_error(Y,y_pred))

72.57565061679861
